In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [13]:
loans_df = pd.read_csv('../data/data.csv')

/var/folders/j3/n1zgv5yd3h9_lk6htkm9rmbc0000gn/T/ipykernel_8886/1797879524.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  loans_df = pd.read_csv('../data/data.csv')


In [14]:
missing_counts = loans_df.isnull().sum()
print("Columns with highest missing values:")
print(missing_counts.sort_values(ascending=False).head(10))

Columns with highest missing values:
member_id                                     2260701
orig_projected_additional_accrued_interest    2252050
hardship_end_date                             2249784
hardship_start_date                           2249784
hardship_type                                 2249784
hardship_reason                               2249784
hardship_status                               2249784
deferral_term                                 2249784
hardship_last_payment_amount                  2249784
hardship_payoff_balance_amount                2249784
dtype: int64


##### Feature selection - Drop columns with too many missing values

In [15]:
missing_threshold = 0.5  # Drop columns with > 50% missing values
cols_to_drop = [col for col in loans_df.columns if loans_df[col].isnull().sum() / len(loans_df) > missing_threshold]
print(f"Dropping {len(cols_to_drop)} columns with more than 50% missing values")
loans_df = loans_df.drop(columns=cols_to_drop)

Dropping 44 columns with more than 50% missing values


##### Handle missing values for key features

In [16]:
if 'emp_length' in loans_df.columns and loans_df['emp_length'].isnull().sum() > 0:
    loans_df['emp_length'].fillna(loans_df['emp_length'].mode()[0], inplace=True)

In [18]:
# DTI - replace with median based on grade
if 'dti' in loans_df.columns and loans_df['dti'].isnull().sum() > 0:
    if 'grade' in loans_df.columns:
        # Group by grade and fill with median DTI for that grade
        loans_df['dti'] = loans_df.groupby('grade')['dti'].transform(
            lambda x: x.fillna(x.median())
        )
    else:
        # Just use overall median
        loans_df['dti'].fillna(loans_df['dti'].median(), inplace=True)

In [19]:
# Credit history features - often missing means "no events"
zero_fill_columns = ['delinq_2yrs', 'inq_last_6mths', 'pub_rec', 
                    'revol_util', 'collections_12_mths_ex_med']
for col in zero_fill_columns:
    if col in loans_df.columns and loans_df[col].isnull().sum() > 0:
        loans_df[col].fillna(0, inplace=True)

In [20]:
# Check if any missing values remain in key modeling features
print("\nRemaining missing values in selected features:")


Remaining missing values in selected features:


In [21]:
# Define target variable (loan status)
# Convert to binary: 1 for default, 0 for fully paid
loans_df['loan_status_binary'] = loans_df['loan_status'].apply(
    lambda x: 1 if x in ['Charged Off', 'Default', 'Late (31-120 days)'] else 0)


In [22]:
features = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade',
          'emp_length', 'home_ownership', 'annual_inc', 'dti',
          'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths',
          'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc']

In [23]:
print(loans_df[features].isnull().sum())

loan_amnt          33
term               33
int_rate           33
installment        33
grade              33
emp_length          0
home_ownership     33
annual_inc         37
dti                33
delinq_2yrs         0
fico_range_low     33
fico_range_high    33
inq_last_6mths      0
open_acc           62
pub_rec             0
revol_bal          33
revol_util          0
total_acc          62
dtype: int64


In [24]:
# Drop rows with missing values in selected features
loans_df = loans_df.dropna(subset=features)

In [25]:
# Verify no missing values remain
print("\nAfter dropping rows, missing values in selected features:")
print(loans_df[features].isnull().sum())


After dropping rows, missing values in selected features:
loan_amnt          0
term               0
int_rate           0
installment        0
grade              0
emp_length         0
home_ownership     0
annual_inc         0
dti                0
delinq_2yrs        0
fico_range_low     0
fico_range_high    0
inq_last_6mths     0
open_acc           0
pub_rec            0
revol_bal          0
revol_util         0
total_acc          0
dtype: int64


In [26]:
# Check how many rows were removed
print(f"\nRemaining rows: {loans_df.shape[0]} (dropped {2260701 - loans_df.shape[0]} rows)")


Remaining rows: 2260639 (dropped 62 rows)


#### Processoing on batches to avoid kernel crash

In [43]:
# First, identify unique values for each categorical feature
# This ensures consistent encoding across chunks
categorical_features = ['term', 'grade', 'emp_length', 'home_ownership']
categorical_values = {}

for feature in categorical_features:
    categorical_values[feature] = loans_df[feature].unique().tolist()

# Process in smaller chunks
chunk_size = 100000
num_chunks = (loans_df.shape[0] // chunk_size) + 1
encoded_chunks = []

# First, create an empty DataFrame with the expected columns
# Get the column structure from a single row
sample = loans_df.iloc[0:1][features]
sample_encoded = pd.get_dummies(sample, drop_first=True)
expected_columns = sample_encoded.columns.tolist()

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, loans_df.shape[0])
    
    # Get the chunk with selected features
    chunk = loans_df.iloc[start_idx:end_idx][features]
    
    # Get dummies with the same structure for each chunk
    chunk_encoded = pd.get_dummies(chunk, drop_first=True)
    
    # Make sure all expected columns exist (this handles potential missing categories in some chunks)
    for col in expected_columns:
        if col not in chunk_encoded.columns:
            chunk_encoded[col] = 0
    
    # Keep only the expected columns and in the same order
    chunk_encoded = chunk_encoded[expected_columns]
    encoded_chunks.append(chunk_encoded)
    
    print(f"Processed chunk {i+1}/{num_chunks}")

# Combine all chunks
loans_df_encoded = pd.concat(encoded_chunks)

# Verify no NaNs
print("\nMissing values in encoded dataframe:")
print(loans_df_encoded.isna().sum().sum())

Processed chunk 1/23
Processed chunk 2/23
Processed chunk 3/23
Processed chunk 4/23
Processed chunk 5/23
Processed chunk 6/23
Processed chunk 7/23
Processed chunk 8/23
Processed chunk 9/23
Processed chunk 10/23
Processed chunk 11/23
Processed chunk 12/23
Processed chunk 13/23
Processed chunk 14/23
Processed chunk 15/23
Processed chunk 16/23
Processed chunk 17/23
Processed chunk 18/23
Processed chunk 19/23
Processed chunk 20/23
Processed chunk 21/23
Processed chunk 22/23
Processed chunk 23/23

Missing values in encoded dataframe:
0


In [46]:
loans_df_encoded.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc'],
      dtype='object')

In [47]:
# Split into training and testing sets
X = loans_df_encoded
y = loans_df['loan_status_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Scale numerical features
scaler = StandardScaler()
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [50]:
# Save processed data
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)